DataPreprocessing

In [1]:
!pip install joblib

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC,SVR
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import RandomForestRegressor
import joblib

Read DataSet

In [3]:
# Read dataset from a excel file
def readFromExcel(file_path):
    df = pd.read_excel(file_path)
    return df

In [4]:
test_data=readFromExcel("TestDatasetExample.xls")

Deal Nan value

In [5]:
# Replace 999 with NaN
# Save this processing dataset to a excel file as FilledEmptyDataset.xlsx
def handleNaN(df):

    df_changed = df.replace(999, np.nan, inplace=False)

    # If NaN shows in pCR and RFS, drop this row
    df_handle = df_changed.dropna(subset=['pCR (outcome)', 'RelapseFreeSurvival (outcome)'])

    # Use median value to replace NaN
    df_filled = df_handle.fillna(df_handle.median(), inplace=False)

    # Reset the index for next step
    df_filled = df_filled.reset_index(drop=True)

    return df_filled

In [6]:
test_data.replace(999,np.nan,inplace=True)
test_data.fillna(test_data.median(), inplace=True)

CheckOutliers

In [7]:
# Find outliers and assign values for them
# Save this processing dataset to a excel file as HandleOutliersDataset.xlsx
def checkOutliers(dataFrame):

    # Extract data after 13th cols
    df = dataFrame.iloc[:, 13:]
    imputer = KNNImputer(n_neighbors=3)
    for column in df.columns:

        # Use boxplots theory to find the outliers and replace them with nan
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3-q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        df.loc[(df[column] > upper_bound) | (df[column] < lower_bound), column] = np.nan

    # Use KNNImputer give value to outliers
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    # Concatenate all columns
    result_df = pd.concat([dataFrame.iloc[:, :13], df], axis=1)

    return result_df

Normilization

In [8]:
# Normalize the dataset
# Save this processing dataset to a excel file as ProcessedDataset.xlsx
# This is the final step for preprocessing
def normData(dataFrame,star):

    # Extract specific columns
    df = pd.concat([dataFrame['Age'], dataFrame.iloc[:, star:]], axis=1)
    result_df=dataFrame.copy()
    for column in df.columns:

        # Make a rule to determine if there is a value out of the range [0, 1] in every column
        is_between_0_and_1 = (df[column] >= 0) & (df[column] <= 1)

        # Use all() to check whether all data in this column follow the rule, if not then normalize this column
        if is_between_0_and_1.all() == False:

            # Use MinMaxScaler to normalize selected column
            scaler = MinMaxScaler()
            df[[column]] = scaler.fit_transform(df[[column]])
            result_df[column]=df[column]
    return result_df

In [9]:
index_of_start = test_data.columns.get_loc('original_shape_Elongation')
print("Index of column of train:", index_of_start)

Index of column of train: 11


In [10]:
test_data=normData(test_data,index_of_start)

In [11]:
display(test_data)

,ID,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002728,0.000000,0,0,0,1,3,3,1.0,0,...,0.194591,0.194591,0.000000,0.001281,1.000000,0.537233,0.002335,0.109755,0.013383,0.002051
1,TRG002649,1.000000,0,0,1,0,2,1,1.0,0,...,0.309999,0.309996,0.137173,0.007253,0.000000,0.000000,0.011285,0.055890,0.003163,0.009553
2,TRG002628,0.433714,0,0,0,1,3,3,1.0,1,...,0.328377,0.328377,1.000000,0.003185,0.859655,1.000000,0.001334,0.101628,0.010844,0.001194


Random Forest feature importance

In [14]:
submit_x=test_data.drop(["ID"],axis=1)

In [13]:
features_list=joblib.load("FeaturelistForRFS")

In [15]:
submit_x=submit_x[features_list]

PCA

In [16]:
display(submit_x)

,original_firstorder_Kurtosis,original_firstorder_Range,original_glszm_SizeZoneNonUniformity,original_firstorder_Maximum,original_firstorder_Variance,original_glszm_ZoneEntropy,original_shape_Elongation,original_firstorder_Skewness,Age,original_firstorder_MeanAbsoluteDeviation,...,original_glrlm_RunLengthNonUniformityNormalized,original_shape_Maximum3DDiameter,original_shape_SurfaceArea,original_shape_Maximum2DDiameterSlice,original_firstorder_Mean,original_ngtdm_Coarseness,original_glrlm_RunPercentage,original_gldm_DependenceNonUniformity,original_glrlm_LongRunHighGrayLevelEmphasis,Proliferation
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.621404,0.177252,0.000000,0.000000,...,0.183692,0.018755,0.211041,0.364869,0.000000,0.002335,0.206807,0.558131,0.982836,3
1,0.927264,0.509502,0.662692,0.731767,0.783658,0.137173,0.733284,1.000000,1.000000,0.700205,...,0.140892,0.000000,0.000000,0.000000,1.000000,0.011285,0.252044,0.000000,0.000000,1
2,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.703667,0.000000,0.433714,1.000000,...,0.160050,1.000000,1.000000,1.000000,0.837693,0.001334,0.205509,1.000000,1.000000,3


In [17]:
# 对标准化后的数据进行PCA变换
pca=joblib.load("PCAforRFS")
principal_components_submit = pca.transform(submit_x)

# 将主成分转换为数据框
pca_submit_x = pd.DataFrame(data=principal_components_submit)

display(pca_submit_x)

c:\Users\DELL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.2.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.506608,1.444578,-1.108581,-0.720206,-0.385492,0.300196,-0.085488,-0.066137,-0.359396,0.555717,0.357331,-0.240002,-0.704297,-0.420818,-0.435929,0.092681
1,-1.287537,-0.053292,0.635132,1.197098,0.413798,-0.054616,0.104868,0.196608,0.022482,-0.481461,-0.507259,0.749390,-0.463490,-0.201204,-0.188765,-0.380845
2,2.573402,-1.311050,1.353390,-1.019074,0.580557,0.350034,-0.620002,0.389310,0.650047,-0.255013,0.365057,-0.235925,-0.131580,0.160263,-0.201269,-0.028830


Cross Validation

对SVR模型调优

In [18]:
svm=joblib.load("SVR")

c:\Users\DELL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVR from version 1.2.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Write predict answer to csv

In [19]:
submit__y=svm.predict(pca_submit_x)
submit_df=pd.DataFrame({"ID":test_data["ID"],"RelapseFreeSurvival (outcome)":submit__y})
submit_df.to_csv("FinalTestRFS.csv")

c:\Users\DELL\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
